# Web Scraping with Pandas

Based on the Data Professor youtube video:
https://youtu.be/JUSFaWkAASI

### Load Dependencies

In [187]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import yfinance as yf

In [188]:
# Import DB user and password
from api_keys_kf import mysql_hostname
from api_keys_kf import mysql_port
from api_keys_kf import mysql_username
from api_keys_kf import mysql_pass

In [196]:
# MySQL specific connection string
database_name = 'etlprojectdb'
table_price = 'price'
table_companies = 'companies'
database_url = f"mysql+mysqlconnector://{mysql_username}:{mysql_pass}@{mysql_hostname}:{mysql_port}/{database_name}"

In [198]:
# Create the engine
from sqlalchemy import create_engine
engine = create_engine(database_url)
connection = engine.connect()

### Create Web Scraping

In [5]:
# Wikipedia List of List of S&P 500 companies
url_wikipedia = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

# Output csv data file name
file_out = os.path.join('List_of_S6P_500_companies.csv')

In [6]:
def load_data(url, table_no):
    html = pd.read_html(url_wikipedia, header=0)
    df = html[table_no]
    return df

In [7]:
companies_list_df = load_data(url_wikipedia, table_no=0)
companies_list_df

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,ABIOMED Inc,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...,...
500,YUM,Yum! Brands Inc,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
501,ZBRA,Zebra Technologies,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
502,ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
503,ZION,Zions Bancorp,reports,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


In [8]:
# Save data as csv
companies_list_df.to_csv(file_out, index=False)

In [9]:
#getting the information for the Comp Table for query

comp_tab_df = companies_list_df

comp_tab_df.drop(columns=['SEC filings', 'Headquarters Location','CIK','Founded'],axis=1, inplace=True)

In [10]:
#remaning columns for loading
comp_tab_df.rename(columns={"Symbol": "comp_tick", "Security": "comp_name", "GICS Sector": "sect_name",
                             "GICS Sub-Industry":"sub_sect_name",
                             "Date first added":"first_trade_date"}, inplace=True)

comp_tab_df.set_index('comp_tick', inplace=True)
comp_tab_df.head(10)

,comp_name,sect_name,sub_sect_name,first_trade_date
comp_tick,,,,
MMM,3M Company,Industrials,Industrial Conglomerates,1976-08-09
ABT,Abbott Laboratories,Health Care,Health Care Equipment,1964-03-31
ABBV,AbbVie Inc.,Health Care,Pharmaceuticals,2012-12-31
ABMD,ABIOMED Inc,Health Care,Health Care Equipment,2018-05-31
ACN,Accenture plc,Information Technology,IT Consulting & Other Services,2011-07-06
ATVI,Activision Blizzard,Communication Services,Interactive Home Entertainment,2015-08-31
ADBE,Adobe Inc.,Information Technology,Application Software,1997-05-05
AMD,Advanced Micro Devices Inc,Information Technology,Semiconductors,2017-03-20
AAP,Advance Auto Parts,Consumer Discretionary,Automotive Retail,2015-07-09


In [11]:
a=0
for index,row in comp_tab_df.iterrows():
    i = len(companies_list_df['first_trade_date'].iloc[a])
    print(i)
    a = a+1

10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
23
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10


TypeError: object of type 'float' has no len()

In [12]:
#spit the row that has two dates
companies_list_df['first_trade_date'].iloc[53].split('(')[0]

'1983-11-30 '

In [13]:
#remove the secon date
companies_list_df['first_trade_date'].iloc[53]=companies_list_df['first_trade_date'].iloc[53].split('(')[0]

In [14]:
#replacing na values with 0
companies_list_df.replace(to_replace="nan", value="0", inplace=False)

companies_list_df.head(505)

,comp_name,sect_name,sub_sect_name,first_trade_date
comp_tick,,,,
MMM,3M Company,Industrials,Industrial Conglomerates,1976-08-09
ABT,Abbott Laboratories,Health Care,Health Care Equipment,1964-03-31
ABBV,AbbVie Inc.,Health Care,Pharmaceuticals,2012-12-31
ABMD,ABIOMED Inc,Health Care,Health Care Equipment,2018-05-31
ACN,Accenture plc,Information Technology,IT Consulting & Other Services,2011-07-06
...,...,...,...,...
YUM,Yum! Brands Inc,Consumer Discretionary,Restaurants,1997-10-06
ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,2019-12-23
ZBH,Zimmer Biomet,Health Care,Health Care Equipment,2001-08-07


In [ ]:
#load to sql
#comp_tab_df.to_sql(table_companies, con = engine, if_exists = 'append', chunksize = 50)


### Data Analysis

In [ ]:
companies_list_df['GICS Sector'].unique()

In [ ]:
# Group Companies by Sector
companies_list_df.groupby('GICS Sector').count()

In [ ]:
# Plot Companies by Sector
ax = companies_list_df.groupby('GICS Sector')['Symbol'].count().plot.bar(figsize=(12,6))
ax.set_xlabel('GICS Sub Industry',size=12)
ax.set_ylabel('Number of Industries',size=12)
ax.set_title(f"Number of Industries by GICS Sector", size=14)
# ax.set_yticks([0, 5, 10, 15, 20])
ax.grid(alpha=.3)
plt.savefig('GICS_Sector.png')

In [ ]:
GICS_Sub_Industry = companies_list_df.groupby('GICS Sub Industry').count().sort_values('Symbol',ascending=False)
GICS_Sub_Industry

In [ ]:
# Plot Companies by Sector
num_industries = 50
ax = GICS_Sub_Industry['Symbol'].head(num_industries).plot.bar(figsize=(15,6))
ax.set_xlabel('GICS Sub Industry',size=12)
ax.set_ylabel('Number of Industries',size=12)
ax.set_title(f"Number of Industries by GICS Sub Industry (Top {num_industries})", size=14)
ax.set_yticks([0, 5, 10, 15, 20])
ax.grid(alpha=.3)
plt.savefig('GICS_Sub_Industry.png')

### Using Yfinance for retrieve stock prices

In [15]:
# List of Stock Symbols
list_symbol_companies = list(companies_list_df.index)



In [16]:
# Retrieve the stock prices
stock_price = yf.download(list_symbol_companies)

[*********************100%***********************]  505 of 505 completed

2 Failed downloads:
- BRK.B: No data found, symbol may be delisted
- BF.B: 1d data not available for startTime=-2208988800 and endTime=1603581126. Only 100 years worth of day granularity data are allowed to be fetched per request.


In [17]:
stock_price.head()

Adj Close                                          ... Volume  \
                   A AAL AAP AAPL ABBV ABC ABMD ABT ACN ADBE  ...   XLNX   
Date                                                          ...          
1962-01-02       NaN NaN NaN  NaN  NaN NaN  NaN NaN NaN  NaN  ...    NaN   
1962-01-03       NaN NaN NaN  NaN  NaN NaN  NaN NaN NaN  NaN  ...    NaN   
1962-01-04       NaN NaN NaN  NaN  NaN NaN  NaN NaN NaN  NaN  ...    NaN   
1962-01-05       NaN NaN NaN  NaN  NaN NaN  NaN NaN NaN  NaN  ...    NaN   
1962-01-08       NaN NaN NaN  NaN  NaN NaN  NaN NaN NaN  NaN  ...    NaN   

                                                          
                  XOM XRAY XRX XYL YUM ZBH ZBRA ZION ZTS  
Date                                                      
1962-01-02   902400.0  NaN NaN NaN NaN NaN  NaN  NaN NaN  
1962-01-03  1200000.0  NaN NaN NaN NaN NaN  NaN  NaN NaN  
1962-01-04  1088000.0  NaN NaN NaN NaN NaN  NaN  NaN NaN  
1962-01-05  1222400.0  NaN NaN NaN NaN NaN  NaN  NaN NaN  
1962-01-08  1388800.0  NaN NaN NaN NaN NaN  NaN  NaN NaN  

[5 rows x 3030 columns]

In [18]:
stock_price['Close']['AAPL']

Date
1962-01-02           NaN
1962-01-03           NaN
1962-01-04           NaN
1962-01-05           NaN
1962-01-08           NaN
                 ...    
2020-10-19    115.980003
2020-10-20    117.510002
2020-10-21    116.870003
2020-10-22    115.750000
2020-10-23    115.040001
Name: AAPL, Length: 14826, dtype: float64

In [38]:
stock_price = stock_price.loc['2010-01-01':]

stock_price.head()

Adj Close                                                       \
                    A       AAL        AAP      AAPL ABBV        ABC  ABMD   
Date                                                                         
2010-01-04  20.579165  4.496876  39.094307  6.604801  NaN  22.606730  8.74   
2010-01-05  20.355618  5.005957  38.861965  6.616219  NaN  22.445440  8.53   
2010-01-06  20.283297  4.798555  39.200817  6.510980  NaN  22.233210  8.40   
2010-01-07  20.257004  4.939965  39.191139  6.498945  NaN  21.876654  8.40   
2010-01-08  20.250422  4.845690  39.346043  6.542150  NaN  22.114357  8.23   

                                             ...     Volume              \
                  ABT        ACN       ADBE  ...       XLNX         XOM   
Date                                         ...                          
2010-01-04  18.022673  33.464951  37.090000  ...  2824700.0  27809100.0   
2010-01-05  17.877064  33.671783  37.700001  ...  3469700.0  30174700.0   
2010-01-06  17.976341  34.029732  37.619999  ...  6329200.0  35044700.0   
2010-01-07  18.125263  33.997910  36.889999  ...  9252700.0  27192100.0   
2010-01-08  18.217920  33.862690  36.689999  ...  5005800.0  24891800.0   

                                                                      \
                 XRAY        XRX XYL        YUM        ZBH      ZBRA   
Date                                                                   
2010-01-04  1051400.0  5112800.0 NaN  2962200.0   782400.0  168800.0   
2010-01-05   763400.0  3255800.0 NaN  3298700.0  1718100.0  168800.0   
2010-01-06  1595100.0  2634300.0 NaN  4178900.0  1277300.0  385300.0   
2010-01-07  1096100.0  4508500.0 NaN  2452400.0  1683500.0  183600.0   
2010-01-08   803600.0  2460100.0 NaN  3772300.0  2149500.0  266500.0   

                            
                  ZION ZTS  
Date                        
2010-01-04   3974600.0 NaN  
2010-01-05   5605500.0 NaN  
2010-01-06  12615200.0 NaN  
2010-01-07  24716800.0 NaN  
2010-01-08   6903000.0 NaN  

[5 rows x 3030 columns]

In [92]:
# Save data as csv
stock_price.to_csv('stock_price.csv', index=False)

In [99]:
clean_stock_price = stock_price

#clean_stock_price.drop(columns=['Adj Close', 'High','Low','Open'],axis=1, inplace=True)
clean_stock_price.drop(columns=['Open'],axis=1, inplace=True)

In [100]:
clean_stock_price.head()

Close                                                   \
                    A   AAL        AAP      AAPL ABBV        ABC  ABMD   
Date                                                                     
2010-01-04  22.389128  4.77  40.380001  7.643214  NaN  26.629999  8.74   
2010-01-05  22.145924  5.31  40.139999  7.656428  NaN  26.440001  8.53   
2010-01-06  22.067240  5.09  40.490002  7.534643  NaN  26.190001  8.40   
2010-01-07  22.038626  5.24  40.480000  7.520714  NaN  25.770000  8.40   
2010-01-08  22.031473  5.14  40.639999  7.570714  NaN  26.049999  8.23   

                                             ...     Volume              \
                  ABT        ACN       ADBE  ...       XLNX         XOM   
Date                                         ...                          
2010-01-04  26.129908  42.070000  37.090000  ...  2824700.0  27809100.0   
2010-01-05  25.918797  42.330002  37.700001  ...  3469700.0  30174700.0   
2010-01-06  26.062737  42.779999  37.619999  ...  6329200.0  35044700.0   
2010-01-07  26.278646  42.740002  36.889999  ...  9252700.0  27192100.0   
2010-01-08  26.412991  42.570000  36.689999  ...  5005800.0  24891800.0   

                                                                      \
                 XRAY        XRX XYL        YUM        ZBH      ZBRA   
Date                                                                   
2010-01-04  1051400.0  5112800.0 NaN  2962200.0   782400.0  168800.0   
2010-01-05   763400.0  3255800.0 NaN  3298700.0  1718100.0  168800.0   
2010-01-06  1595100.0  2634300.0 NaN  4178900.0  1277300.0  385300.0   
2010-01-07  1096100.0  4508500.0 NaN  2452400.0  1683500.0  183600.0   
2010-01-08   803600.0  2460100.0 NaN  3772300.0  2149500.0  266500.0   

                            
                  ZION ZTS  
Date                        
2010-01-04   3974600.0 NaN  
2010-01-05   5605500.0 NaN  
2010-01-06  12615200.0 NaN  
2010-01-07  24716800.0 NaN  
2010-01-08   6903000.0 NaN  

[5 rows x 1010 columns]

In [181]:
price_df = pd.DataFrame(columns = ["comp_tick","date","close_price","volume","currency"])

In [183]:
row_df = pd.DataFrame(columns = ["comp_tick","close_price","volume","currency"])
count = 505
tick = 1
comp = 0
for row in clean_stock_price.itertuples():
#     name = list_symbol_companies[comp]
#     close= row[tick]
#     vol = row[count+tick]
    data = row
    date = row[0]
    row_dict=[]
    for i in list_symbol_companies:
#         row_df['comp_tick'] = list_symbol_companies[comp]
#         row_df['close_price'] = data[tick]
#         row_df['volume'] = data[count+tick]
#         row_df['currency'] = "USD"
        row_dict.append({'comp_tick':list_symbol_companies[comp], 'date':date,'close_price':data[tick], 'volume':  data[count+tick],'currency':"USD"})
        #print(row_dict)
        
        tick = 1+tick
        comp = comp+1
    tick = 1
    comp = 0
    price_df = price_df.append(row_dict)
    print(date)
        #comp = comp+1

#         row_df['comp_tick'] = name
#         row_df['close_price'] = clean_stock_price['Close'][name]
#         row_df['volume'] = clean_stock_price['Volume'][name]
#         row_df['currency'] = "USD"
#         price_df = price_df.append(row_df)

2010-01-04 00:00:00
2010-01-05 00:00:00
2010-01-06 00:00:00
2010-01-07 00:00:00
2010-01-08 00:00:00
2010-01-11 00:00:00
2010-01-12 00:00:00
2010-01-13 00:00:00
2010-01-14 00:00:00
2010-01-15 00:00:00
2010-01-19 00:00:00
2010-01-20 00:00:00
2010-01-21 00:00:00
2010-01-22 00:00:00
2010-01-25 00:00:00
2010-01-26 00:00:00
2010-01-27 00:00:00
2010-01-28 00:00:00
2010-01-29 00:00:00
2010-02-01 00:00:00
2010-02-02 00:00:00
2010-02-03 00:00:00
2010-02-04 00:00:00
2010-02-05 00:00:00
2010-02-08 00:00:00
2010-02-09 00:00:00
2010-02-10 00:00:00
2010-02-11 00:00:00
2010-02-12 00:00:00
2010-02-16 00:00:00
2010-02-17 00:00:00
2010-02-18 00:00:00
2010-02-19 00:00:00
2010-02-22 00:00:00
2010-02-23 00:00:00
2010-02-24 00:00:00
2010-02-25 00:00:00
2010-02-26 00:00:00
2010-03-01 00:00:00
2010-03-02 00:00:00
2010-03-03 00:00:00
2010-03-04 00:00:00
2010-03-05 00:00:00
2010-03-08 00:00:00
2010-03-09 00:00:00
2010-03-10 00:00:00
2010-03-11 00:00:00
2010-03-12 00:00:00
2010-03-15 00:00:00
2010-03-16 00:00:00


2011-08-18 00:00:00
2011-08-19 00:00:00
2011-08-22 00:00:00
2011-08-23 00:00:00
2011-08-24 00:00:00
2011-08-25 00:00:00
2011-08-26 00:00:00
2011-08-29 00:00:00
2011-08-30 00:00:00
2011-08-31 00:00:00
2011-09-01 00:00:00
2011-09-02 00:00:00
2011-09-06 00:00:00
2011-09-07 00:00:00
2011-09-08 00:00:00
2011-09-09 00:00:00
2011-09-12 00:00:00
2011-09-13 00:00:00
2011-09-14 00:00:00
2011-09-15 00:00:00
2011-09-16 00:00:00
2011-09-19 00:00:00
2011-09-20 00:00:00
2011-09-21 00:00:00
2011-09-22 00:00:00
2011-09-23 00:00:00
2011-09-26 00:00:00
2011-09-27 00:00:00
2011-09-28 00:00:00
2011-09-29 00:00:00
2011-09-30 00:00:00
2011-10-03 00:00:00
2011-10-04 00:00:00
2011-10-05 00:00:00
2011-10-06 00:00:00
2011-10-07 00:00:00
2011-10-10 00:00:00
2011-10-11 00:00:00
2011-10-12 00:00:00
2011-10-13 00:00:00
2011-10-14 00:00:00
2011-10-17 00:00:00
2011-10-18 00:00:00
2011-10-19 00:00:00
2011-10-20 00:00:00
2011-10-21 00:00:00
2011-10-24 00:00:00
2011-10-25 00:00:00
2011-10-26 00:00:00
2011-10-27 00:00:00


2013-04-08 00:00:00
2013-04-09 00:00:00
2013-04-10 00:00:00
2013-04-11 00:00:00
2013-04-12 00:00:00
2013-04-15 00:00:00
2013-04-16 00:00:00
2013-04-17 00:00:00
2013-04-18 00:00:00
2013-04-19 00:00:00
2013-04-22 00:00:00
2013-04-23 00:00:00
2013-04-24 00:00:00
2013-04-25 00:00:00
2013-04-26 00:00:00
2013-04-29 00:00:00
2013-04-30 00:00:00
2013-05-01 00:00:00
2013-05-02 00:00:00
2013-05-03 00:00:00
2013-05-06 00:00:00
2013-05-07 00:00:00
2013-05-08 00:00:00
2013-05-09 00:00:00
2013-05-10 00:00:00
2013-05-13 00:00:00
2013-05-14 00:00:00
2013-05-15 00:00:00
2013-05-16 00:00:00
2013-05-17 00:00:00
2013-05-20 00:00:00
2013-05-21 00:00:00
2013-05-22 00:00:00
2013-05-23 00:00:00
2013-05-24 00:00:00
2013-05-28 00:00:00
2013-05-29 00:00:00
2013-05-30 00:00:00
2013-05-31 00:00:00
2013-06-03 00:00:00
2013-06-04 00:00:00
2013-06-05 00:00:00
2013-06-06 00:00:00
2013-06-07 00:00:00
2013-06-10 00:00:00
2013-06-11 00:00:00
2013-06-12 00:00:00
2013-06-13 00:00:00
2013-06-14 00:00:00
2013-06-17 00:00:00


2014-11-24 00:00:00
2014-11-25 00:00:00
2014-11-26 00:00:00
2014-11-28 00:00:00
2014-12-01 00:00:00
2014-12-02 00:00:00
2014-12-03 00:00:00
2014-12-04 00:00:00
2014-12-05 00:00:00
2014-12-08 00:00:00
2014-12-09 00:00:00
2014-12-10 00:00:00
2014-12-11 00:00:00
2014-12-12 00:00:00
2014-12-15 00:00:00
2014-12-16 00:00:00
2014-12-17 00:00:00
2014-12-18 00:00:00
2014-12-19 00:00:00
2014-12-22 00:00:00
2014-12-23 00:00:00
2014-12-24 00:00:00
2014-12-26 00:00:00
2014-12-29 00:00:00
2014-12-30 00:00:00
2014-12-31 00:00:00
2015-01-02 00:00:00
2015-01-05 00:00:00
2015-01-06 00:00:00
2015-01-07 00:00:00
2015-01-08 00:00:00
2015-01-09 00:00:00
2015-01-12 00:00:00
2015-01-13 00:00:00
2015-01-14 00:00:00
2015-01-15 00:00:00
2015-01-16 00:00:00
2015-01-20 00:00:00
2015-01-21 00:00:00
2015-01-22 00:00:00
2015-01-23 00:00:00
2015-01-26 00:00:00
2015-01-27 00:00:00
2015-01-28 00:00:00
2015-01-29 00:00:00
2015-01-30 00:00:00
2015-02-02 00:00:00
2015-02-03 00:00:00
2015-02-04 00:00:00
2015-02-05 00:00:00


2016-07-13 00:00:00
2016-07-14 00:00:00
2016-07-15 00:00:00
2016-07-18 00:00:00
2016-07-19 00:00:00
2016-07-20 00:00:00
2016-07-21 00:00:00
2016-07-22 00:00:00
2016-07-25 00:00:00
2016-07-26 00:00:00
2016-07-27 00:00:00
2016-07-28 00:00:00
2016-07-29 00:00:00
2016-08-01 00:00:00
2016-08-02 00:00:00
2016-08-03 00:00:00
2016-08-04 00:00:00
2016-08-05 00:00:00
2016-08-08 00:00:00
2016-08-09 00:00:00
2016-08-10 00:00:00
2016-08-11 00:00:00
2016-08-12 00:00:00
2016-08-15 00:00:00
2016-08-16 00:00:00
2016-08-17 00:00:00
2016-08-18 00:00:00
2016-08-19 00:00:00
2016-08-22 00:00:00
2016-08-23 00:00:00
2016-08-24 00:00:00
2016-08-25 00:00:00
2016-08-26 00:00:00
2016-08-29 00:00:00
2016-08-30 00:00:00
2016-08-31 00:00:00
2016-09-01 00:00:00
2016-09-02 00:00:00
2016-09-06 00:00:00
2016-09-07 00:00:00
2016-09-08 00:00:00
2016-09-09 00:00:00
2016-09-12 00:00:00
2016-09-13 00:00:00
2016-09-14 00:00:00
2016-09-15 00:00:00
2016-09-16 00:00:00
2016-09-19 00:00:00
2016-09-20 00:00:00
2016-09-21 00:00:00


2018-02-26 00:00:00
2018-02-27 00:00:00
2018-02-28 00:00:00
2018-03-01 00:00:00
2018-03-02 00:00:00
2018-03-05 00:00:00
2018-03-06 00:00:00
2018-03-07 00:00:00
2018-03-08 00:00:00
2018-03-09 00:00:00
2018-03-12 00:00:00
2018-03-13 00:00:00
2018-03-14 00:00:00
2018-03-15 00:00:00
2018-03-16 00:00:00
2018-03-19 00:00:00
2018-03-20 00:00:00
2018-03-21 00:00:00
2018-03-22 00:00:00
2018-03-23 00:00:00
2018-03-26 00:00:00
2018-03-27 00:00:00
2018-03-28 00:00:00
2018-03-29 00:00:00
2018-04-02 00:00:00
2018-04-03 00:00:00
2018-04-04 00:00:00
2018-04-05 00:00:00
2018-04-06 00:00:00
2018-04-09 00:00:00
2018-04-10 00:00:00
2018-04-11 00:00:00
2018-04-12 00:00:00
2018-04-13 00:00:00
2018-04-16 00:00:00
2018-04-17 00:00:00
2018-04-18 00:00:00
2018-04-19 00:00:00
2018-04-20 00:00:00
2018-04-23 00:00:00
2018-04-24 00:00:00
2018-04-25 00:00:00
2018-04-26 00:00:00
2018-04-27 00:00:00
2018-04-30 00:00:00
2018-05-01 00:00:00
2018-05-02 00:00:00
2018-05-03 00:00:00
2018-05-04 00:00:00
2018-05-07 00:00:00


2019-10-11 00:00:00
2019-10-14 00:00:00
2019-10-15 00:00:00
2019-10-16 00:00:00
2019-10-17 00:00:00
2019-10-18 00:00:00
2019-10-21 00:00:00
2019-10-22 00:00:00
2019-10-23 00:00:00
2019-10-24 00:00:00
2019-10-25 00:00:00
2019-10-28 00:00:00
2019-10-29 00:00:00
2019-10-30 00:00:00
2019-10-31 00:00:00
2019-11-01 00:00:00
2019-11-04 00:00:00
2019-11-05 00:00:00
2019-11-06 00:00:00
2019-11-07 00:00:00
2019-11-08 00:00:00
2019-11-11 00:00:00
2019-11-12 00:00:00
2019-11-13 00:00:00
2019-11-14 00:00:00
2019-11-15 00:00:00
2019-11-18 00:00:00
2019-11-19 00:00:00
2019-11-20 00:00:00
2019-11-21 00:00:00
2019-11-22 00:00:00
2019-11-25 00:00:00
2019-11-26 00:00:00
2019-11-27 00:00:00
2019-11-29 00:00:00
2019-12-02 00:00:00
2019-12-03 00:00:00
2019-12-04 00:00:00
2019-12-05 00:00:00
2019-12-06 00:00:00
2019-12-09 00:00:00
2019-12-10 00:00:00
2019-12-11 00:00:00
2019-12-12 00:00:00
2019-12-13 00:00:00
2019-12-16 00:00:00
2019-12-17 00:00:00
2019-12-18 00:00:00
2019-12-19 00:00:00
2019-12-20 00:00:00


In [184]:
#del price_df
price_df.head(100)

,comp_tick,date,close_price,volume,currency
0,A,2010-01-04,22.389128,3815500.0,USD
1,AAL,2010-01-04,4.770000,9837300.0,USD
2,AAP,2010-01-04,40.380001,1701700.0,USD
3,AAPL,2010-01-04,7.643214,493729600.0,USD
4,ABBV,2010-01-04,NaN,NaN,USD
...,...,...,...,...,...
95,CINF,2010-01-04,26.500000,1261400.0,USD
96,CL,2010-01-04,41.424999,4874200.0,USD
97,CLX,2010-01-04,61.430000,1014300.0,USD
98,CMA,2010-01-04,30.500000,1438400.0,USD


In [185]:
price_df_new = price_df
price_df_new.set_index('comp_tick', inplace=True)
price_df_new.head(10)


,date,close_price,volume,currency
comp_tick,,,,
A,2010-01-04,22.389128,3815500.0,USD
AAL,2010-01-04,4.770000,9837300.0,USD
AAP,2010-01-04,40.380001,1701700.0,USD
AAPL,2010-01-04,7.643214,493729600.0,USD
ABBV,2010-01-04,NaN,NaN,USD
ABC,2010-01-04,26.629999,2455900.0,USD
ABMD,2010-01-04,8.740000,149100.0,USD
ABT,2010-01-04,26.129908,10829000.0,USD
ACN,2010-01-04,42.070000,3650100.0,USD


In [200]:
price_df_new.head(100)

,date,close_price,volume,currency
comp_tick,,,,
A,2010-01-04,22.389128,3815500.0,USD
AAL,2010-01-04,4.770000,9837300.0,USD
AAP,2010-01-04,40.380001,1701700.0,USD
AAPL,2010-01-04,7.643214,493729600.0,USD
ABBV,2010-01-04,NaN,NaN,USD
...,...,...,...,...
CINF,2010-01-04,26.500000,1261400.0,USD
CL,2010-01-04,41.424999,4874200.0,USD
CLX,2010-01-04,61.430000,1014300.0,USD


In [199]:
#Load to the sql database

#price_df.to_sql(table_price, con = engine, if_exists = 'append', chunksize = 1000)